# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed(overall passing %73.6) compared to schools with smaller budgets (<\$585 per student)(overall passing %95).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_school = len(school_data_complete['school_name'].unique())
total_students = '{:,}'.format(len(school_data_complete['Student ID']))
total_budget = '${:,.2f}'.format(school_data['budget'].sum())
avg_math_score = round(school_data_complete['math_score'].mean(),6)
avg_reading_score = round(school_data_complete['reading_score'].mean(),5)
overall_passing_rate = (avg_math_score + avg_reading_score)/2
percent_passing_math = round(len(school_data_complete.loc[school_data_complete['math_score'] >= 70])*100/len(school_data_complete),6)
percent_passing_read = round(len(school_data_complete.loc[school_data_complete['reading_score'] >= 70])*100/len(school_data_complete),6)
dist_summary_df = pd.DataFrame({'Total Schools':[total_school],
                                'Total Students':[total_students],
                                'Total Budget':[total_budget],
                                'Average Math Score':[avg_math_score],
                                'Average Reading Score':[avg_reading_score],
                                '% Passing Math':[percent_passing_math],
                                '% Passing Reading':[percent_passing_read],
                                '% Overall Passing Rate':[overall_passing_rate]})
dist_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:
grouped_school_data_complete = school_data_complete.groupby('school_name').agg({'type':'min',
                                                                       'Student ID':'count',
                                                                       'budget':'max',
                                                                       'math_score':'mean',
                                                                       'reading_score':'mean', 'size':'min'
                                                                      }).rename(columns={'type':'School Type',
                                                                                'Student ID':'Total Students',
                                                                                'budget':'Total School Budget',
                                                                                'math_score':'Average Math Score',
                                                                                'reading_score':'Average Reading Score'})

passing_math_data = school_data_complete.loc[school_data_complete['math_score'] >= 70]
passing_read_data = school_data_complete.loc[school_data_complete['reading_score'] >= 70]

grouped_passing_math_data = passing_math_data.groupby('school_name').agg({'Student ID':'count'}).rename(columns={'Student ID':'Math Pass Count'})
grouped_passing_read_data = passing_read_data.groupby('school_name').agg({'Student ID':'count'}).rename(columns={'Student ID':'Read Pass Count'})

#grouped_school_data_complete
grouped_school_summary = grouped_school_data_complete.merge(grouped_passing_math_data, how='left', on=["school_name"]).merge(grouped_passing_read_data, how='left', on=["school_name"])
grouped_school_summary['% Passing Math'] = round(grouped_school_summary['Math Pass Count']*100/grouped_school_summary['Total Students'],6)
grouped_school_summary['% Passing Reading'] = round(grouped_school_summary['Read Pass Count']*100/grouped_school_summary['Total Students'],6)
grouped_school_summary['% Overall Passing Rate'] = ((round(grouped_school_summary['% Passing Math'],6) + round(grouped_school_summary['% Passing Reading'],6))/2).map('{:.6f}'.format)
grouped_school_summary['Per Student Budget'] = (grouped_school_summary['Total School Budget']/grouped_school_summary['Total Students']).map("${:,.2f}".format)
grouped_school_summary.index.name = ''
school_summary = grouped_school_summary[['School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']].sort_values(['% Overall Passing Rate'], ascending=False)
school_summary['Total School Budget'] = school_summary['Total School Budget'].map('${:,.2f}'.format)
school_summary.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270271
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
worst_school = school_summary.sort_values(['% Overall Passing Rate'])
worst_school.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363853
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
nine_grade = school_data_complete.loc[school_data_complete['grade']=='9th']\
                                 .groupby('school_name')\
                                 .agg({'math_score':'mean', 'reading_score':'mean'})\
                                 .rename(columns={'math_score':'9th', 'reading_score':'reading_score_9'})
ten_grade = school_data_complete.loc[school_data_complete['grade']=='10th']\
                                 .groupby('school_name')\
                                 .agg({'math_score':'mean', 'reading_score':'mean'})\
                                 .rename(columns={'math_score':'10th', 'reading_score':'reading_score_10'})
eleven_grade = school_data_complete.loc[school_data_complete['grade']=='11th']\
                                 .groupby('school_name')\
                                 .agg({'math_score':'mean', 'reading_score':'mean'})\
                                 .rename(columns={'math_score':'11th','reading_score':'reading_score_11'})
twelve_grade = school_data_complete.loc[school_data_complete['grade']=='12th']\
                                 .groupby('school_name')\
                                 .agg({'math_score':'mean', 'reading_score':'mean'})\
                                 .rename(columns={'math_score':'12th','reading_score':'reading_score_12'})
grouped_grade = nine_grade.merge(ten_grade, how='left', on='school_name')\
                          .merge(eleven_grade, how='left', on='school_name')\
                          .merge(twelve_grade, how='left', on='school_name')
grouped_grade.index.name=''
grouped_grade['9th'] = grouped_grade['9th'].map('{:.6f}'.format)
grouped_grade['10th'] = grouped_grade['10th'].map('{:.6f}'.format)
grouped_grade['11th'] = grouped_grade['11th'].map('{:.6f}'.format)
grouped_grade['12th'] = grouped_grade['12th'].map('{:.6f}'.format)
grouped_grade_math = grouped_grade[['9th','10th','11th','12th']]
grouped_grade_math

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
grouped_grade_read = grouped_grade[['reading_score_9','reading_score_10','reading_score_11','reading_score_12']]
grouped_grade_read = grouped_grade_read.rename(columns={'reading_score_9':'9th',
                                                'reading_score_10':'10th',
                                                'reading_score_11':'11th',
                                                'reading_score_12':'12th'})
grouped_grade_read['9th'] = grouped_grade_read['9th'].map('{:.6f}'.format)
grouped_grade_read['10th'] = grouped_grade_read['10th'].map('{:.6f}'.format)
grouped_grade_read['11th'] = grouped_grade_read['11th'].map('{:.6f}'.format)
grouped_grade_read['12th'] = grouped_grade_read['12th'].map('{:.6f}'.format)
grouped_grade_read

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_summary_bin = pd.DataFrame(school_summary[['Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']])
school_summary_bin['Per Student Budget'] = school_summary_bin['Per Student Budget'].str.replace(',','')
school_summary_bin['Per Student Budget'] = school_summary_bin['Per Student Budget'].str.replace('$','')
school_summary_bin['Per Student Budget'] = school_summary_bin['Per Student Budget'].astype(float)
school_summary_bin['Spending Ranges (Per Student)'] = pd.cut(school_summary_bin['Per Student Budget'], spending_bins, labels =group_names)

grouped_school_summary_bin = school_summary_bin.groupby('Spending Ranges (Per Student)').agg({'Average Math Score':'mean',
                                                                                              'Average Reading Score':'mean',
                                                                                              '% Passing Math':'mean',
                                                                                              '% Passing Reading':'mean'})

grouped_school_summary_bin['% Overall Passing Rate']=(grouped_school_summary_bin['% Passing Math']+grouped_school_summary_bin['% Passing Reading'])/2
grouped_school_summary_bin

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065573
$615-645,79.079225,81.891436,75.668212,86.106570,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [8]:
# bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_size_bin = pd.DataFrame(grouped_school_summary[['size','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']])
school_size_bin['School Size'] = pd.cut(school_size_bin['size'], size_bins, labels =group_names)

grouped_school_size_bin = school_size_bin.groupby('School Size').agg({'Average Math Score':'mean',
                                                                                              'Average Reading Score':'mean',
                                                                                              '% Passing Math':'mean',
                                                                                              '% Passing Reading':'mean'})

grouped_school_size_bin['% Overall Passing Rate']=(grouped_school_size_bin['% Passing Math']+grouped_school_size_bin['% Passing Reading'])/2
grouped_school_size_bin


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099436,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [9]:
grouped_school_summary.head()
grouped_school_type = grouped_school_summary.groupby('School Type').agg({'Average Math Score':'mean',
                                                                                              'Average Reading Score':'mean',
                                                                                              '% Passing Math':'mean',
                                                                                              '% Passing Reading':'mean'})

grouped_school_type['% Overall Passing Rate']=(grouped_school_type['% Passing Math']+grouped_school_type['% Passing Reading'])/2
grouped_school_type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
